<a href="https://colab.research.google.com/github/jsstar522/hunkim_ML/blob/master/GAN/03_1D_Gaussian_Distribution_GAN_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaussian 분포 GAN 모델 적용

input 데이터를 Gaussian 분포로 만든 다음 GAN 모델을 통해 최대한 input 분포와 비슷하게 뽑아내겠습니다. 

## Input 데이터 생성과 데이터 입력

### Discriminator
Discriminator에 들어가는 input 데이터는 평균 4, 표준편차 1.25의 분포를 가지고 있는 n개의 데이터입니다. Tensor는 (1, n)의 형태를 가지고 있습니다.

### Generator
Generator 모델에 들어가는 input 데이터 (noise)는 0~1의 값을 가지고 있는 무작위 n개의 데이터입니다. 그리고 layer를 통과한 값은 Discriminator로 들어가므로 Discriminator의 input인 1 x n의 형태를 띄어야 합니다.

In [69]:
import numpy as np
import tensorflow as tf


data_mean = 4
data_stddev = 1.25

g_input_size = 1
g_hidden_size = 256
g_output_size = 1

d_input_size = 100
d_hidden_size = 256
d_output_size = 1

minibatch_size = d_input_size

## discriminator 정규분포로 뽑기 (1 x n)
def get_distribution_sampler(mu, sigma):
    return lambda n: tf.convert_to_tensor(np.random.normal(mu, sigma, (1, n)))

## generator 무작위 uniform 뽑기 ( m x n )
def get_generator_input_sampler():
    return lambda m, n : tf.convert_to_tensor(np.random.uniform(0, 1, (m, n)))


test_d = get_distribution_sampler(data_mean, data_stddev)
sess = tf.Session()
print(test_d(d_input_size))
print(sess.run(test_d(d_input_size)))

test_g = get_generator_input_sampler()
print(test_g(minibatch_size, g_input_size))
print(sess.run(test_g(minibatch_size, g_input_size)))

Tensor("Const_21:0", shape=(1, 100), dtype=float64)
[[3.56201999 2.91720754 3.2461213  3.53618347 4.32595812 4.0344969
  3.36136328 5.06961147 4.26005992 2.35930401 2.52253729 5.10616605
  3.36409819 6.55941718 4.59743423 3.52897069 2.7398972  5.71246107
  2.85082859 6.3360992  6.09603312 3.88712081 5.08315305 4.92549988
  4.31420518 4.1922948  3.61684676 5.46483484 3.77688295 5.04399559
  7.22468822 2.60433647 5.21002754 4.58196723 4.15753422 4.94395047
  5.83101531 3.47839572 3.6401248  3.4076198  5.1772122  3.90984997
  2.39486091 3.55844388 5.00666411 3.08378989 5.90240914 3.90770068
  3.77823279 5.19817219 4.9416321  3.01368067 3.10479161 3.44060943
  4.32433781 3.4197083  3.96652456 4.19125024 3.40164208 5.21437072
  3.48777905 4.46757548 3.82684589 5.30608454 5.70211158 2.7473225
  4.74844075 1.11733646 4.12962366 4.35204567 1.78801713 2.68070302
  2.2768711  2.31877883 2.77709222 2.46275101 6.04991288 3.57838386
  3.24175202 5.93514636 2.2758573  3.52392904 3.78733321 4.5165604

위의 함수로 생성된 input 값 두개는 다음과 같이 활용됩니다.

![GAN input data](figure/1.jpeg)

## Generator 와 Discriminator

이제 Generator와 Discriminator를 만들겠습니다.

In [47]:
class Discriminator:
    def __init__(self, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()
        self.D_W1 = tf.Variable(tf.random_normal([input_size, hidden_size], stddev=0.01))
        self.D_W2 = tf.Variable(tf.random_normal([hidden_size, output_size], stddev=0.01))
        self.D_b1 = tf.Variable(tf.zeros([hidden_size]))
        self.D_b2 = tf.Variable(tf.zeros([output_size]))

    def forward(self, inputs):
        hidden = tf.nn.relu(tf.matmul(inputs, self.D_W1) + self.D_b1)
        output = tf.nn.sigmoid(tf.matmul(hidden, self.D_w) + self.D_b2)
        return output
        

class Generator:
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.G_W1 = tf.Variable(tf.random_normal([input_size, hidden_size], stddev=0.01))
        self.G_W2 = tf.Variable(tf.random_normal([hidden_size, output_size], stddev=0.01))
        self.G_b1 = tf.Variable(tf.zeros([hidden_size]))
        self.G_b2 = tf.Variable(tf.zeros([output_size]))

    def forward(self, inputs):
        hidden = tf.nn.relu(tf.matmul(inputs, self.G_W1) + self.G_b1)
        output = tf.nn.sigmoid(tf.matmul(hidden, self.G_w) + self.G_b2)
        return output

## input size , hidden size = 256, output size = 1(True or False)
D = Discriminator(input_size= d_input_size, hidden_size=d_hidden_size, output_size=d_output_size)
## input size = 1, hidden size = 256, output size = 1(1 x n)
G = Generator(input_size=g_input_size, hidden_size=g_hidden_size, output_size=g_output_size)
        